<a href="https://colab.research.google.com/github/eelcofolkertsma/BIX-samples/blob/main/Track%20dependencies.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

This is a python script to collect dependencies between types in xml library